In [115]:
import numpy as np
import pandas as pd

In [116]:
df = pd.read_csv("data_full/persons_fixed.csv")
df

,Unnamed: 0,household_id,person_id,SEX,AGE,SCH,ESR,NAICSP02,SOCP10,SW_UNSPLIT_IND,SW_OCCUP,SW_SPLIT_IND
0,0,1,1,2,58,1,6,0,0,0,0,0
1,1,2,2,1,35,1,3,0,9,0,9,0
2,2,3,3,2,86,1,6,0,0,0,0,0
3,3,4,4,1,83,1,6,0,0,0,0,0
4,4,4,5,2,83,1,6,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13645997,13645997,5643151,13645998,1,50,1,1,6,11,6,11,12
13645998,13645998,5643151,13645999,2,49,1,1,10,7,10,7,17
13645999,13645999,5643151,13646000,1,23,2,1,3,17,3,17,5
13646000,13646000,5643151,13646001,1,20,2,1,16,17,16,17,23


In [150]:
df["ptype"] = 0

In [151]:
df["ESR"].value_counts()

1    6150745
6    3821596
0    2940113
3     538785
2     181667
4      12918
5        178
Name: ESR, dtype: int64

Employment status recode (ESR) \
b .N/A (less than 16 years old) \
1 .Civilian employed, at work \
2 .Civilian employed, with a job but not at work (e.g., teachers during summer, maternity leave) \
3 .Unemployed \
4 .Armed forces, at work \
5 .Armed forces, with a job but not at work (see 2) \
6 .Not in labor force

In [152]:
df["SCH"].value_counts()

1    9639038
2    2914756
3     592841
0     499367
Name: SCH, dtype: int64

School enrollment \
b .N/A (less than 3 years old) \
1 .No, has not attended in the last 3 months \
2 .Yes, public school or public college \
3 .Yes, private school, private college, or home school

In [153]:
# ptype == 1 are full-time workers
# are temporarily absent workers full-time?
df["ptype"] = np.where(df["ESR"].isin([1, 2, 4, 5]), 1, df["ptype"])

In [154]:
# ptype == 2 are part-time workers (no equivalency)


In [155]:
# ptype == 3, univeristy students
# assume all people above 19 and in school are in university; mtc seems ot be more detailed
df["ptype"] = np.where((df["SCH"].isin([2, 3])) & (df["AGE"] >= 19), 3, df["ptype"])

In [156]:
# ptype == 4, non-working adult
# include non-working 16-65 year olds only (seems to be what mtc has except with 16 instead of 17 as the lower bound)
df["ptype"] = np.where(((df["AGE"] >= 16) & (df["AGE"] <= 65) & (df["ESR"].isin([3, 6]))), 4, df["ptype"])

In [157]:
# ptype == 5, retirees
# above 65 and not in labor force/unemployed, follows mtc
df["ptype"] = np.where((df["AGE"] >= 65) & (df["ESR"].isin([3, 6])), 5, df["ptype"])

In [158]:
# ptype == 6, driving-age child who is in school
# driving age as in permit?
# 16-18 year olds in school (not mtc, which seems to be more detailed)
df["ptype"] = np.where((df["AGE"] >= 16) & (df["AGE"] <= 18) & (df["SCH"].isin([2, 3])), 6, df["ptype"])

In [159]:
# ptype == 7, pre-driving age child who is in school
# 0-15 year olds in school
# mtc defines as 6-15 year olds, regardless of school status
df["ptype"] = np.where((df["AGE"] <= 15) & (df["AGE"] >= 6), 7, df["ptype"])

In [160]:
# ptype == 8, pre-driving age who are too young for school
# AGE <= 5 (kindergarten starts at 6), which is what mtc has
# some 3 year olds are in school
df["ptype"] = np.where(df["AGE"] <= 5, 8, df["ptype"])

In [161]:
# everybody covered
df["ptype"].value_counts()

1    5649579
4    2339873
7    1915192
5    1655128
8    1024921
6     540973
3     520336
Name: ptype, dtype: int64

In [162]:
mtc_persons = pd.read_csv("../prototype_mtc/data/persons.csv")
mtc_persons

,PERID,household_id,age,RELATE,ESR,GRADE,PNUM,PAUG,DDP,sex,WEEKS,HOURS,MSP,POVERTY,EARNS,pagecat,pemploy,pstudent,ptype,padkid
0,25671,25671,47,1,6,0,1,0,0,1,0,0,6,39,0,6,3,3,4,2
1,25675,25675,27,1,6,7,1,0,0,2,52,40,2,84,7200,5,3,2,3,2
2,25678,25678,30,1,6,0,1,0,0,2,0,0,6,84,0,5,3,3,4,2
3,25683,25683,23,1,6,0,1,0,0,1,0,0,6,1,0,4,3,3,4,2
4,25684,25684,52,1,6,0,1,0,0,1,0,0,6,94,0,7,3,3,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8207,7554848,2863513,68,22,6,0,1,0,0,1,0,0,4,0,0,8,3,3,5,2
8208,7554855,2863520,68,22,6,0,1,0,0,1,0,0,4,0,0,8,3,3,5,2
8209,7554859,2863524,93,22,6,0,1,0,0,2,0,0,2,0,0,9,3,3,5,2
8210,7554887,2863552,76,22,6,0,1,0,0,2,0,0,2,0,0,8,3,3,5,2


In [163]:
mtc_persons["ptype"].value_counts()

1    3027
5    1299
4    1215
2    1038
3     640
7     505
8     347
6     141
Name: ptype, dtype: int64

In [164]:
# 7 is inflated, 3 is deflated
df["ptype"].value_counts()

1    5649579
4    2339873
7    1915192
5    1655128
8    1024921
6     540973
3     520336
Name: ptype, dtype: int64

In [165]:
df.to_csv("data_full/persons_fixed_cdap.csv")

In [8]:
temp = pd.read_csv("data_cleanup/t10_converted/persons-updated.csv")

In [18]:
temp

,HH_ID,PERS_ID,SEX,AGEP,SCH,ESR,NAICSP02,socp10,SW_UNSPLIT_IND,SW_OCCUP,SW_SPLIT_IND,pstudent,pemploy
0,1,1,2,58,1,6,0,0,0,0,23,3,3
1,2,1,1,35,1,3,0,9,0,9,23,3,0
2,3,1,2,86,1,6,0,0,0,0,23,3,3
3,4,1,1,83,1,6,0,0,0,0,23,3,3
4,4,2,2,83,1,6,0,0,0,0,23,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13645997,5643151,1,1,50,1,1,6,11,6,11,12,3,0
13645998,5643151,2,2,49,1,1,10,7,10,7,17,3,0
13645999,5643151,3,1,23,2,1,3,17,3,17,5,2,0
13646000,5643151,4,1,20,2,1,16,17,16,17,23,2,0
